In [119]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils,layers,datasets
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [120]:
import numpy as np
import matplotlib.pyplot as plt

In [121]:
cifar=datasets=datasets.cifar100
(train_images,train_labels),(test_images,test_labels)=cifar.load_data()

In [122]:

print(train_images.shape,train_labels.shape,test_images.shape,test_labels.shape)
print(test_labels)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
[[49]
 [33]
 [72]
 ...
 [51]
 [42]
 [70]]


In [123]:
import cv2
import os

def load_data(path,row,column):

    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    tmp = 1
    each_img_paths=[path+"\\"+name for name in os.listdir(path)]
    #print(each_img_paths)
    for idx in range(len(each_img_paths)):
        image_path=each_img_paths[idx]
        #print(image_path)
        img=cv2.imread(image_path,cv2.IMREAD_COLOR)
        cv2.imshow('img', img)
        img=cv2.resize(img, (row,column), interpolation = cv2.INTER_AREA)
        if tmp==4:
            tmp=1
            x_test.append(img)
            y_test.append([1])
        else:
            tmp=tmp+1
            x_train.append(img)
            y_train.append([[0],[1]])
    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)


In [124]:
path = "D:\\hanyang\\git\\workplace\\Tensorflow_study\\wine"
x_train,y_train,x_test,y_test= load_data(path,32,32)

In [125]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(307, 32, 32, 3) (307, 2, 1) (102, 32, 32, 3) (102, 1)


In [126]:
X_train=np.concatenate((x_train,train_images),axis=0)
    
Y_train=np.concatenate((y_train,np.zeros((50000,1))),axis=0)
X_test=np.concatenate((x_test,test_images),axis=0)
Y_test=np.concatenate((y_test,np.zeros((10000,1))),axis=0)

print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)      

seed = 100
np.random.seed(seed)

randomize = np.arange(len(X_train))
np.random.shuffle(randomize)
X_train = X_train[randomize]
Y_train = Y_train[randomize]

randomize = np.arange(len(X_test))
np.random.shuffle(randomize)
X_test = X_test[randomize]
Y_test = Y_test[randomize]

X_train=X_train.astype(float)/255.0
X_test=X_test.astype(float)/255.0
Y_train=to_categorical(Y_train,2)
Y_test=to_categorical(Y_test,2)

print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)      


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64,(3,3),padding='same',activation='relu',input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='softmax'))
    
    return model
    

In [127]:
m=make_model()
m.summary()
m.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 4096)            

In [129]:
epochs = 35
early_stopping =EarlyStopping(monitor='val_loss',patience=10)

In [130]:
history = m.fit(
    X_train,Y_train,
    epochs=epochs,
    validation_data=(X_test,Y_test),
    callbacks=[early_stopping]
)

Train on 50307 samples, validate on 10102 samples
Epoch 1/35
50307/50307 [==============================] - 117s 2ms/step - loss: 0.3328 - accuracy: 0.9930 - val_loss: 0.3805 - val_accuracy: 0.9899
Epoch 2/35
50307/50307 [==============================] - 131s 3ms/step - loss: 0.0550 - accuracy: 0.9939 - val_loss: 0.1733 - val_accuracy: 0.9899
Epoch 3/35
50307/50307 [==============================] - 137s 3ms/step - loss: 0.0587 - accuracy: 0.9938 - val_loss: 0.0883 - val_accuracy: 0.9899
Epoch 4/35
50307/50307 [==============================] - 114s 2ms/step - loss: 0.0424 - accuracy: 0.9939 - val_loss: 0.0644 - val_accuracy: 0.9899
Epoch 5/35
50307/50307 [==============================] - 129s 3ms/step - loss: 0.0429 - accuracy: 0.9939 - val_loss: 0.0569 - val_accuracy: 0.9899
Epoch 6/35
50307/50307 [==============================] - 124s 2ms/step - loss: 0.0381 - accuracy: 0.9939 - val_loss: 0.0586 - val_accuracy: 0.9899
Epoch 7/35
50307/50307 [==============================] - 113s

In [131]:
from keras.models import load_model
m.save('cnn_epoch'+str(epochs)+"seed_"+str(seed)+"_model.h5")

y_vloss=history.history['val_loss']
y_loss=history.history['loss']

x_len=np.arange(len(y_loss))

plt.plot(x_len,y_vloss,marker=".",c="red",label='val_set_loss')
plt.plot(x_len,y_loss,marker=".",c="blue",label='train_set_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

NameError: name 'seed' is not defined